In [ ]:
import hamlib.hamlib_snippets as hs

import numpy as np
import scipy as sp
import matplotlib.pylab as plt
import warnings

from qiskit.circuit.random import random_circuit
from qiskit import QuantumCircuit, QuantumRegister
from qiskit.circuit.library import PauliEvolutionGate
from qiskit.synthesis import LieTrotter
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager

from qiskit.providers.fake_provider import GenericBackendV2
from qiskit_ibm_runtime import QiskitRuntimeService, SamplerV2 as Sampler

from qiskit_addon_sqd.counts import counts_to_arrays
from qiskit_addon_sqd.qubit import solve_qubit
from collections import Counter

warnings.filterwarnings("ignore")

In [ ]:
# fname = "hamlib/test/ham/test_ham.hdf5"
# key = "test_ham"
fname = "hamlib/condensedmatter/tfim/tfim.hdf5"
key = "/graph-1D-grid-pbc-qubitnodes_Lx-12_h-1"
# key = "/graph-2D-grid-nonpbc-qubitnodes_Lx-4_Ly-4_h-1"

H_op = hs.read_qiskit_hdf5(fname, key)
n_qubits = H_op.num_qubits

A = np.array(H_op)
eigenvalues, eigenvectors = np.linalg.eigh(A)
gse = min(eigenvalues)
print("The ground state energy is ", gse)
plot_gse = True

# gse = -25.393496754735867
# plot_gse = True

# plot_gse = False

In [ ]:
# Set parameters for quantum Krylov algorithm
krylov_dim = 4  # size of krylov subspace
# dt = .3
dt = np.pi / np.linalg.norm(H_op, ord=2)
num_trotter_steps = 12

In [ ]:
# Prep the reference state for evolution
 
qc_state_prep = QuantumCircuit(n_qubits)
qc_state_prep.h(0)
for i in range(n_qubits - 1):
    qc_state_prep.cx(i, i+1)

# qc_state_prep = QuantumCircuit(n_qubits)
# for i in range(n_qubits):
#     qc_state_prep.h(i)

# qc_state_prep = random_circuit(n_qubits, 4)

In [ ]:
evol_gate = PauliEvolutionGate(
    H_op, time=(dt / num_trotter_steps), synthesis=LieTrotter(reps=num_trotter_steps)
)  # `U` operator
 
qr = QuantumRegister(n_qubits)
qc_evol = QuantumCircuit(qr)
qc_evol.append(evol_gate, qargs=qr)
 
circuits = []
for rep in range(krylov_dim):
    circ = qc_state_prep.copy()
 
    # Repeating the `U` operator to implement U^0, U^1, U^2, and so on, for power Krylov space
    for _ in range(rep):
        circ.compose(other=qc_evol, inplace=True)
 
    circ.measure_all()
    circuits.append(circ)

In [ ]:
circuits[1].decompose().draw("mpl", fold=-1)

In [ ]:
circuits[2].decompose().draw("mpl", fold=-1)

In [ ]:
backend = GenericBackendV2(num_qubits=n_qubits+1)
pm = generate_preset_pass_manager(backend=backend, optimization_level=3)
isa_circuits = pm.run(circuits=circuits)

In [ ]:
sampler = Sampler(mode=backend)
job = sampler.run(isa_circuits, shots=1_000_000)

In [ ]:
counts_all = [job.result()[k].data.meas.get_counts() for k in range(krylov_dim)]

In [ ]:
counts_cumulative = []
for i in range(krylov_dim):
    counter = Counter()
    for d in counts_all[: i + 1]:
        counter.update(d)
 
    counts = dict(counter)
    counts_cumulative.append(counts)

In [ ]:
# # Filters out bitstrings that do not have specified number (`num_ones`) of `1` bits.
# def postselect_counts(counts, num_ones):
#     filtered_counts = {}
#     for bitstring, freq in counts.items():
#         if bitstring.count("1") == num_ones:
#             filtered_counts[bitstring] = freq
 
#     return filtered_counts

In [ ]:
scipy_kwargs = {"k": 3, "which": "SA"}
 
ground_state_energies = []
for idx, counts in enumerate(counts_cumulative):
    # counts = postselect_counts(counts, num_ones=n_qubits // 2)
    bitstring_matrix, probs = counts_to_arrays(counts=counts)
 
    eigenvals, eigenstates = solve_qubit(
        bitstring_matrix, H_op, verbose=False, **scipy_kwargs
    )
    gs_en = np.min(eigenvals)
    ground_state_energies.append(gs_en)

In [ ]:
plt.plot(
    range(1, krylov_dim + 1),
    ground_state_energies,
    color="blue",
    linestyle="-.",
    label="estimate",
)
if plot_gse:
    plt.plot(
        range(1, krylov_dim + 1),
        [gse] * krylov_dim,
        color="red",
        linestyle="-",
        label="exact",
    )
plt.xticks(range(1, krylov_dim + 1), range(1, krylov_dim + 1))
plt.legend()
plt.xlabel("Krylov space dimension")
plt.ylabel("Energy")
plt.title(
    "Estimating Ground state energy with Sample-based Krylov Quantum Diagonalization"
)
plt.show()

In [ ]:
print("Differences")
for i in range(krylov_dim):
    print("dim. ", i + 1, ": ", gse - ground_state_energies[i])

*This code was part of the work done as part of the Qiskit Advocate Mentorship Programme (QAMP) 2025 project No.: 31.*\
*Mentors: Dr. Soham Pal, Dr. Shiplu Sarker,*\
*Mentees: Abdullah Afzal, Michael Papadopoulos, Gayathree M. Vinod.*\
*This notebook was prepared by Michael Papadopoulos.*